In [ ]:
def attention_block_2d(x, g, inter_channel):
    # theta_x(?,g_height,g_width,inter_channel)

    theta_x = Conv2D(filters=inter_channel, kernel_size=(1, 1), strides=(1, 1),padding='same')(x)

    # phi_g(?,g_height,g_width,inter_channel)

    phi_g = Conv2D(filters=inter_channel, kernel_size=(1, 1), strides=(1, 1),padding='same')(g)

    # f(?,g_height,g_width,inter_channel)

    f = Activation('relu')(add([theta_x, phi_g]))

    # psi_f(?,g_height,g_width,1)

    psi_f = Conv2D(filters=1, kernel_size=(1, 1), strides=(1, 1),padding='same')(f)

    rate = Activation('sigmoid')(psi_f)

    # rate(?,x_height,x_width)

    # att_x(?,x_height,x_width,x_channel)

    att_x = multiply([x, rate])

    return att_x
    
def attention_up_and_concate(down_layer, layer):

    in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2),interpolation = 'nearest')(down_layer)  

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4)

    concate = concatenate([up, layer], axis=3)

    return concate

def down_block(x, filt):
    cn = Conv2D(filters = filt, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
               use_bias=True, kernel_initializer='glorot_normal')(x)
    d1 =  Dropout(0.2)(cn)
    cn1 = Conv2D(filters = filt,  kernel_size=(3, 3), strides=1, padding='same', activation='relu',
               use_bias=True, kernel_initializer='glorot_normal')(d1)
    p = MaxPooling2D((2, 2), strides=(2, 2),padding='same')(cn1)
    return cn1, p

def up_block(x, skip, filt):
    concat = attention_up_and_concate(x, skip)
    cnn = Conv2D(filters=filt, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
               use_bias=True, kernel_initializer='glorot_normal')(concat)
    d2 =  Dropout(0.2)(cnn)     
    cnn1 = Conv2D(filters=filt, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
               use_bias=True, kernel_initializer='glorot_normal')(d2)
    return cnn1

def bottleneck(x, filt):
    cnt = Conv2D(filters=filt, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
               use_bias=True, kernel_initializer='glorot_normal')(x)
    d3 = Dropout(0.2)(cnt)
    cnt1 = Conv2D(filters=filt, kernel_size=(3, 3), strides=1, padding='same', activation='relu',
               use_bias=True, kernel_initializer='glorot_normal')(d3)
    return cnt1


def att_UNet():
    f = [64, 128, 256, 512, 1024]
    #inputs = keras.layers.Input((image_size, image_size, 3))
    inputs= Input((512,512,3))
    
    # p0 = inputs
    c1, p1 = down_block(inputs, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    pd1 = Dropout(0.2)(p4)   
    bn = bottleneck(pd1, f[4])
    pd2 = Dropout(0.2)(bn)   
    u1 = up_block(pd2, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = Conv2D(filters=1, kernel_size=(1,1), strides=(1,1), padding='same', activation='sigmoid')(u4)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
model_attUnet = att_UNet()
adam = keras.optimizers.Adam(lr = 0.001)
model_attUnet.compile(optimizer = "adam" , loss = 'binary_crossentropy' , metrics = ["acc"])
model_attUnet.summary()